In [ ]:
# After importing the data, there are

# 8 measurements in the file
# The steps are, for each of the measurements:
# 1. Plot each measurement
# 2. Perform baseline subtractions
# 3. Determine the peaks
# 4. For each peak:
#    a. Find the the maximum value (beginning/end of peak?)
#    b. The integral of the peak
#    c. The time at which the peak reaches its maximum value.  (??)
# 5. Output the calculated data to a file, and plot it data.

 # Import the data file, subtract baselines

In [1]:
import pandas as pd
import lab_toolbox

# Import data
df = pd.read_csv("Example_input.txt", header=25)
df.set_index('Time/s', inplace=True)
df.columns = df.columns.str.strip()  # Strip whitespace from the columns

# Create a structure to hold all of the measurements (i1/A, i2/A, ...)
measurements = {}
for col in df.columns:
    x = lab_toolbox.Measurement(col, df[col])
    measurements[x.name] = x
    
lab_toolbox.filter_baseline_interactively(measurements)

FigureWidget({
    'data': [{'line': {'color': '#636EFA'},
              'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, ..., #a3a7e4,
                                   #a3a7e4, #a3a7e4],
                         'line': {'color': 'DarkSlateGrey', 'width': 0},
                         'size': [0, 0, 0, ..., 0, 0, 0]},
              'mode': 'lines+markers',
              'name': 'i1/A',
              'type': 'scattergl',
              'uid': 'ee745f9b-ab04-4142-afa9-7b07dac4e580',
              'x': array([5.000e+00, 1.000e+01, 1.500e+01, ..., 6.600e+03, 6.605e+03, 6.610e+03]),
              'xaxis': 'x',
              'y': array([3.138e-05, 2.905e-05, 2.843e-05, ..., 2.395e-05, 2.401e-05, 2.364e-05]),
              'yaxis': 'y'},
             {'hoverinfo': 'skip',
              'line': {'color': '#FF6692'},
              'mode': 'lines',
              'name': 'spline_i1/A',
              'type': 'scatter',
              'uid': '599c124e-6eec-426b-b04c-f3ea724b60c0',
              'x':

# Select start:end of peak

In [2]:
samples = 3  # Get user's input here
lab_toolbox.integrate_peaks_interactively(measurements, samples=3)

FigureWidget({
    'data': [{'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, ..., #a3a7e4,
                                   #a3a7e4, #a3a7e4],
                         'line': {'color': 'DarkSlateGrey', 'width': 0},
                         'size': [0, 0, 0, ..., 0, 0, 0]},
              'mode': 'lines+markers',
              'name': 'i1/A',
              'type': 'scattergl',
              'uid': '9c2c81fa-4545-416f-80ac-b588e65fee22',
              'x': array([5.000e+00, 1.000e+01, 1.500e+01, ..., 6.600e+03, 6.605e+03, 6.610e+03]),
              'xaxis': 'x',
              'y': array([ 0.00000000e+00, -6.02217723e-07, -6.38058008e-09, ...,  1.86266449e-07,
                           2.46205520e-07, -1.23930143e-07]),
              'yaxis': 'y'},
             {'marker': {'color': [#a3a7e4, #a3a7e4, #a3a7e4, ..., #a3a7e4,
                                   #a3a7e4, #a3a7e4],
                         'line': {'color': 'DarkSlateGrey', 'width': 0},
                         'size': [0, 0

# Find the peaks in the data